In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [18]:
data = pd.read_csv('./data/train.csv', encoding='windows-1251')
#test = pd.read_csv('./data/test.csv', encoding='windows-1251')

In [3]:
data.head()

,Interval,Date,OrderDate,ClientID,ChannelID,OrderID,MaterialID,GroupID,Cluster,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit
0,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328810.0,61.0,NaN,0,1.0,Обычная доставка,0,1
1,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3281258.0,30.0,NaN,0,2.0,Обычная доставка,0,1
2,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3210734.0,10.0,NaN,0,1.0,Обычная доставка,0,1
3,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328848.0,61.0,NaN,0,2.0,Обычная доставка,0,1
4,12-14.,07/10/2018,05/10/2018,94112406,2,90102091007,3347801.0,17.0,NaN,0,10.0,Обычная доставка,0,1


In [19]:
order_cnt = data[['OrderID', 'OrderCnt']].groupby('OrderID').sum().reset_index()

In [20]:
data = data.drop(['ClientID', 'MaterialID', 'GroupID', 'OrderCnt'], axis=1)

In [21]:
data = data.drop_duplicates()

In [23]:
interval_time = data['Interval'].apply(lambda x: int(x.split('-')[1][:-1]) - int(x.split('-')[0]))

In [24]:
order_weekday = data['OrderDate'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())
weekday = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())

In [55]:
interval_low = data['Interval'].apply(lambda x: int(x.split('-')[0])).apply(lambda x: x if x != 0 else 24)
interval_high = data['Interval'].apply(lambda x: int(x.split('-')[1][:-1])).apply(lambda x: x if x != 0 else 24)
interval_time = interval_high - interval_low
interval_avg = (interval_high + interval_low) / 2

In [56]:
morning = list(range(6, 12))
day = list(range(12, 18))
evening = list(range(18, 24))
night = [24] + list(range(1, 6))

In [57]:
morning = interval_avg.isin(morning)
day = interval_avg.isin(day)
evening = interval_avg.isin(evening)
night = interval_avg.isin(night)

In [64]:
data['interval_time'] = interval_time
data['order_weekday'] = order_weekday
data['weekday'] = weekday
data['interval_low'] = interval_low
data['interval_high'] = interval_high
data['interval_time'] = interval_time
data['interval_avg'] = interval_avg
data['morning'] = morning.apply(lambda x: 1 if x else 0)
data['day'] = day.apply(lambda x: 1 if x else 0)
data['evening'] = evening.apply(lambda x: 1 if x else 0)
data['night'] = night.apply(lambda x: 1 if x else 0)

In [72]:
data['is_order_weekend'] = data['order_weekday'].apply(lambda x: 1 if x in [5,6] else 0)
data['is_weekend'] = data['weekday'].apply(lambda x: 1 if x in [5,6] else 0)

In [60]:
data['Cluster'] = data['Cluster'].fillna('foo')

In [62]:
data['count_edit'] -= 1

In [69]:
label_cols = ['DeliveryType', 'Cluster']

for col in label_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [74]:
data = data.drop(['Interval', 'Date', 'OrderDate'], axis=1)

In [75]:
data.head()

,ChannelID,OrderID,Cluster,CancelFlag,DeliveryType,prepay,count_edit,interval_time,order_weekday,weekday,interval_low,interval_high,interval_avg,morning,day,evening,night,is_order_weekend,is_weekend
0,2,90102063002,23,0,1,0,0,2,1,2,14,16,15.0,0,1,0,0,0,0
4,2,90102091007,23,0,1,0,0,2,4,6,12,14,13.0,0,1,0,0,0,1
9,2,90102092000,23,0,0,0,0,2,4,4,19,21,20.0,0,0,1,0,0,0
19,2,90102103017,23,0,1,0,0,2,4,4,22,24,23.0,0,0,1,0,0,0
28,2,90102104012,23,1,1,0,0,8,0,2,10,18,14.0,0,1,0,0,0,0


In [76]:
data.to_csv('./data/data_preprocessed.csv', index=False)